In [50]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline 
from sklearn.pipeline import make_pipeline
from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier

In [6]:
df = pd.read_csv('data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [46]:
df.shape[0]

569

In [83]:
print("Dados NaN:")
print(df.isna().sum())

Dados NaN:
id                         0
diagnosis                  0
radius_mean                0
texture_mean               0
perimeter_mean             0
area_mean                  0
smoothness_mean            0
compactness_mean           0
concavity_mean             0
concave points_mean        0
symmetry_mean              0
fractal_dimension_mean     0
radius_se                  0
texture_se                 0
perimeter_se               0
area_se                    0
smoothness_se              0
compactness_se             0
concavity_se               0
concave points_se          0
symmetry_se                0
fractal_dimension_se       0
radius_worst               0
texture_worst              0
perimeter_worst            0
area_worst                 0
smoothness_worst           0
compactness_worst          0
concavity_worst            0
concave points_worst       0
symmetry_worst             0
fractal_dimension_worst    0
dtype: int64


In [7]:
X = df.drop('diagnosis',axis=1)
y = df['diagnosis']

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [92]:

pipeline_rf = make_pipeline(
    SimpleImputer(strategy='mean'),
    MinMaxScaler(),
    RandomForestClassifier()
)

# Crie um pipeline para o modelo Decision Tree:
pipeline_dt = make_pipeline(
    SimpleImputer(strategy='mean'),
    MinMaxScaler(),
    DecisionTreeClassifier()
)

# Crie um pipeline para o modelo Decision Tree:
pipeline_ab = make_pipeline(
    SimpleImputer(strategy='mean'),
    MinMaxScaler(),
    AdaBoostClassifier()
)

# Treine e avalie cada pipeline separadamente:
pipelines = [
    ('random_forest', pipeline_rf),
    ('decision_tree', pipeline_dt),
    ('ada_boost', pipeline_ab)
]

for model_name, pipeline in pipelines:
    # Treine o pipeline usando os dados de treinamento:
    pipeline.fit(X_train, y_train)
    
    # Avalie o desempenho do modelo no pipeline usando os dados de teste:
    accuracy = pipeline.score(X_test, y_test)
    print(f"Acurácia do modelo {model_name}: {accuracy}")

Acurácia do modelo random_forest: 0.9649122807017544
Acurácia do modelo decision_tree: 0.9298245614035088
Acurácia do modelo ada_boost: 0.9649122807017544


In [79]:
voting_clf = VotingClassifier(
    estimators=pipelines,
    voting='hard'
)

# Treinar o VotingClassifier
voting_clf.fit(X_train, y_train)

# Avaliar o VotingClassifier
accuracy = voting_clf.score(X_test, y_test)
print("Acurácia do VotingClassifier:", accuracy)


Acurácia do VotingClassifier: 0.9649122807017544


In [93]:
# Criar o classificador base
base_classifier = DecisionTreeClassifier()

# Criar o AdaBoostClassifier
ada_clf = AdaBoostClassifier()

# Criar o pipeline
pipe1 = Pipeline([
    ('si', SimpleImputer(strategy='mean')),
    ('min_max', MinMaxScaler()),
    ('clf', ada_clf)
])

# Definir os hiperparâmetros que queremos ajustar
parameters = {
    'clf__learning_rate': [0.1, 0.05, 0.01, 0.5],
    'clf__n_estimators': [50, 100, 200]
}

# Criar o GridSearchCV
grid_search = GridSearchCV(pipe1, parameters, cv=5)

# Treinar o GridSearchCV
grid_search.fit(X_train, y_train)

# Melhores hiperparâmetros
print("Melhores hiperparâmetros encontrados:")
print(grid_search.best_params_)

# Avaliar o modelo com os melhores hiperparâmetros
accuracy = grid_search.score(X_test, y_test)
print("Acurácia do modelo com melhores hiperparâmetros:", accuracy)


Melhores hiperparâmetros encontrados:
{'clf__learning_rate': 0.5, 'clf__n_estimators': 200}
Acurácia do modelo com melhores hiperparâmetros: 0.956140350877193
